In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/DSAIL')

import os
os.chdir('/content/drive/My Drive/DSAIL')



Mounted at /content/drive


In [4]:
from sklearn.metrics import roc_auc_score
import numpy as np
import torch
from torch import optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [74]:
file_path = "ml-100k/u.data"

train_data = pd.read_csv(file_path, sep='\t', header=None)
train_data.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
# timestamp에 따라 정렬
train_data = train_data.sort_values(['user_id', 'timestamp']).reset_index(drop=True)
print(train_data)

       user_id  movie_id  rating  timestamp
0            1       168       5  874965478
1            1       172       5  874965478
2            1       165       5  874965518
3            1       156       4  874965556
4            1       196       5  874965677
...        ...       ...     ...        ...
99995      943       449       1  888693158
99996      943       229       2  888693158
99997      943       230       1  888693158
99998      943       228       3  888693158
99999      943       234       3  888693184

[100000 rows x 4 columns]


In [ ]:
dataset = train_data
gb_ui = dataset.groupby('user_id')
print(gb_ui.apply(print))

In [ ]:
gb_ui['movie id'].apply(print)

In [76]:
file_path = "ml-100k/u.user"

users = pd.read_csv(file_path, sep='|', header=None)
users.columns = ['user_id', 'age', 'gender', 'occupation', 'zip code']
users = users.drop(['zip code', 'occupation'], axis=1)

# occupation도 있으면 얘 embedding도 학습시켜야 하나봐,,, 빼자
users['gender'] = users['gender'].map({'F': 1, 'M': 0})
print(users)

     user_id  age  gender
0          1   24       0
1          2   53       1
2          3   23       0
3          4   24       0
4          5   33       1
..       ...  ...     ...
938      939   26       1
939      940   32       0
940      941   20       0
941      942   48       1
942      943   22       0

[943 rows x 3 columns]


In [59]:
items = pd.read_csv('ml-100k/u.item', sep="|", encoding='latin-1', header=None)
items.columns = ['movie_id', 'movie title' ,'release date','video release date', 'IMDb URL',
                 'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy',
                 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
                 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = items.drop(['release date','movie title' , 'video release date', 'IMDb URL', 'unknown'], axis=1)
# 안쓰는애들 버리기
print(items)

      movie_id  Action  Adventure  Animation  Children's  Comedy  Crime  \
0            1       0          0          1           1       1      0   
1            2       1          1          0           0       0      0   
2            3       0          0          0           0       0      0   
3            4       1          0          0           0       1      0   
4            5       0          0          0           0       0      1   
...        ...     ...        ...        ...         ...     ...    ...   
1677      1678       0          0          0           0       0      0   
1678      1679       0          0          0           0       0      0   
1679      1680       0          0          0           0       0      0   
1680      1681       0          0          0           0       1      0   
1681      1682       0          0          0           0       0      0   

      Documentary  Drama  Fantasy  Film-Noir  Horror  Musical  Mystery  \
0               0      0 

In [61]:
genres_list = pd.read_csv('ml-100k/u.genre',
                     sep="|", header=None, usecols=[0])[0].tolist()
genres_list = [genre for genre in genres_list if genre != 'unknown']
print(genres_list, len(genres_list))

['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'] 18


In [95]:
wide_data = pd.merge(train_data, users[['user_id', 'age']], on='user_id', how='left')
wide_data = wide_data.drop(['user_id', 'movie_id','rating'], axis = 1)
print(wide_data)

       timestamp  age
0      874965478   24
1      874965478   24
2      874965518   24
3      874965556   24
4      874965677   24
...          ...  ...
99995  888693158   22
99996  888693158   22
99997  888693158   22
99998  888693158   22
99999  888693184   22

[100000 rows x 2 columns]


In [100]:
deep_data = pd.merge(train_data, users[['user_id', 'gender']], on='user_id', how='left')
columns_to_drop = set(['movie_id'] + genres_list)
deep_data = pd.merge(deep_data, items[columns_to_drop], on='movie_id', how='left')
y = deep_data['rating']
deep_data =deep_data.drop(['rating', 'timestamp'], axis = 1)
print(deep_data)

       user_id  movie_id  gender  Sci-Fi  Children's  Action  Documentary  \
0            1       168       0       0           0       0            0   
1            1       172       0       1           0       1            0   
2            1       165       0       0           0       0            0   
3            1       156       0       0           0       0            0   
4            1       196       0       0           0       0            0   
...        ...       ...     ...     ...         ...     ...          ...   
99995      943       449       0       1           0       1            0   
99996      943       229       0       1           0       1            0   
99997      943       230       0       1           0       1            0   
99998      943       228       0       1           0       1            0   
99999      943       234       0       0           0       1            0   

       Thriller  Romance  Musical  ...  Horror  Crime  Film-Noir  Comedy  \

<ipython-input-100-249a75051705>:3: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  deep_data = pd.merge(deep_data, items[columns_to_drop], on='movie_id', how='left')


In [102]:
print(users['user_id'].nunique() , items['movie_id'].nunique())
# genre: 18 # users: 942 movie: 1682
'''
 continuous(Wide)
 age / timestamp

 categoricatl(Deep)
 genre(18) / gender (1) : 0 or 1
 '''


943 1682


'\n continuous(Wide)\n age / timestamp\n\n categoricatl(Deep)\n genre(18) / gender (1) : 0 or 1\n '

In [97]:
from sklearn.model_selection import train_test_split

deep_train, deep_test,wide_train, wide_test, y_train, y_test = train_test_split(deep_data, wide_data, y, test_size=0.3, random_state=42)


In [103]:
class WD(nn.Module):
  def __init__(self, y_train = y_train, y_test = y_test, d = 32, wide_train = wide_train, genres_list= genres_list, deep_train = deep_train, lr = 1e-3, wide_test = wide_test, deep_test = deep_test, epochs = 30):
      super(WD, self,).__init__()
      '''
      Wide & Deep Learning
      기존의 task는 prediction이 0 또는 1인데, 이 코드에서는 그냥 Rating Prediction (1 ~ 5 classifier 달기)
      or
      Timestamp 따라서 movie 하나씩 shift 하면 next movie prediction도 가능

      WIde: Continous
        age + timestamp [100000 rows x 2 columns]

      Deep: Categorical [100000 rows x 20 columns]
        genre + gender
        19
       '''

      self.dimension = d
      self.epochs = epochs
      self.lr = lr
      self.genres_list = genres_list

      self.optim = optim.Adagrad(model.parameters(), lr= self.lr)
      self.criterion = nn.MSELoss()

      # embedding for Deep
      self.embedded_user = nn.Embedding(num_embeddings = 943 , embedding_dim = 32)
      self.embedded_movie = nn.Embedding(num_embeddings = 1682 , embedding_dim = 32)

      self.len_deep = 32 + 32 + 1 + 18

      # 이것도 actie function 지우고 그냥 regression으로 처리하면 바로 1로 해도 될것 같고 1~5로 하려면 classifier 달면될듯
      # next movie prediction으로 가려면,,, 너무 커짐
      self.last_layer = nn.Embedding(self.len_deep, 1)

      # 다음부터는 그냥 Dataset 정의해서 쓰자,,, 왜 그러는지 이제 알겠다,,,,,,,,,,,,,,,,,,,,,,,,,,,,, ooops
      self.wide_train = wide_train
      self.wide_test = wide_test
      self.deep_train = deep_train
      self.deep_test = deep_test
      self.y_train = y_train
      self.y_test = y_test

      ## For Layer
      self.activation = torch.ReLU()

      self.deep_linear_stack = nn.Sequential(
          nn.Linear(self.len_deep, 1024),
          nn.ReLU(),
          nn.Linear(1024, 512),
          nn.ReLU(),
          nn.Linear(512, 256),
          nn.ReLU()
      )

  def forward(self, i):
    # 사실 batch 마다 돌렸어야 함,,

    embedded_user = self.embedded_user(self.deep_train['user_id'][i])
    embedded_movie = self.embedded_movie(self.deep_train['movie_id'][i])
    genres = self.deep_train[self.genres_list].iloc[i].values
    gender = self.deep_train['gender'].iloc[i]

    categorical_input = torch.cat([embedded_user, embedded_movie, genres, gender.unsqueeze(0)], dim=0)

    deep = self.deep_forward(categorical_input)
    # 여기서는 두 개라서 그냥 직접했지만 보통은 dataset에 저장할 때 이미 cross - product 처리해서 하는듯
    wide = self.wide_train['age'][i] * self.wide_train['timestamp'][i] # 1
    widenDeep = torch.cat([wide, deep])

    prediction = self.output_layer(widenDeep)

    return prediction

  def deep_forward(self, categorical_input):
    final_layer = self.deep_linear_stack(categorical_input)
    return self.activation(final_layer)

  def fit(self):

    for epoch in range(self.epochs):
      train_loss = 0
      self.optim.zero_grad()
      for i in range(len(self.y_train)):
        prediction = self.forward(i)
        loss = self.criterion(y_train[i], prediction)

        loss.backward()
        self.optim.step()

        train_loss += loss.item()

      if epoch % 5 == 0:
        print(f"epoch: {epoch}, train_loss: {train_loss}")


In [ ]:
from torch.utils.data import Dataset, DataLoader

class WD(nn.Module):
    def __init__(self, d=32, wide_train=None, genres_list=genres_list, deep_train=None, lr=1e-3, epochs=30):
        super(WD, self).__init__()

        self.dimension = d
        self.epochs = epochs
        self.lr = lr
        self.genres_list = genres_list

        # embedding for Deep
        self.embedded_user = nn.Embedding(num_embeddings=943, embedding_dim=32)
        self.embedded_movie = nn.Embedding(num_embeddings=1682, embedding_dim=32)

        self.len_deep = 32 + 32 + 1 + 18

        # Regression output layer
        self.output_layer = nn.Linear(self.len_deep, 1)

        self.optim = optim.Adagrad(self.parameters(), lr=self.lr)

        self.deep_linear_stack = nn.Sequential(
            nn.Linear(self.len_deep, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU()
        )

    def forward(self, user_id, movie_id, genres, gender, age, timestamp):
        embedded_user = self.embedded_user(user_id)
        embedded_movie = self.embedded_movie(movie_id)

        categorical_input = torch.cat([embedded_user, embedded_movie, genres, gender.unsqueeze(0)], dim=0)

        deep = self.deep_forward(categorical_input)
        wide = age * timestamp
        widen_deep = torch.cat([wide, deep], dim=1)

        # Regression output
        prediction = self.output_layer(widen_deep)
        return prediction

    def deep_forward(self, categorical_input):
        final_layer = self.deep_linear_stack(categorical_input)
        return final_layer

    def fit(self, train_loader):
        criterion = nn.MSELoss()


        for epoch in range(self.epochs):
            train_loss = 0

            for batch in train_loader:
                user_id, movie_id, genres, gender, age, timestamp, y_train = batch

                self.optim.zero_grad()

                predictions = self.forward(user_id, movie_id, genres, gender, age, timestamp)
                loss = criterion(predictions, y_train.view(-1, 1))

                loss.backward()
                self.optim.step()

                train_loss += loss.item()

            self.eval()
            with torch.no_grad():
                for batch in test_loader:
                    user_id, movie_id, genres, gender, age, timestamp, y_test = batch
                    predictions = self.forward(user_id, movie_id, genres, gender, age, timestamp)
                    loss = criterion(predictions, y_test.view(-1, 1))
                    test_loss += loss.item()
            if epoch % 5 == 0:
                print(f"epoch: {epoch}, train_loss: {train_loss / len(train_loader)}, test_loss: {test_loss / len(test_loader)}")



In [ ]:
class MovieDataset(Dataset):
    def __init__(self, wide_data, deep_data, train_data):
        self.user_ids = deep_data['user_ids']
        self.movie_ids = deep_data['movie_ids']
        self.genres =deep_data['genres']
        self.gender = deep_data['genres']
        self.age = wide_data['age']
        self.timestamp = wide_data['timestamp']
        self.ratings = train_data['ratings']

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return (
            self.user_ids[idx],
            self.movie_ids[idx],
            self.genres[idx],
            self.gender[idx],
            self.age[idx],
            self.timestamp[idx],
            self.ratings[idx]
        )
# 뭐 요론 느낌...

## 의문인점
그럼 Wide 모델에서 cross-product 하는 애들은 임의로 고르면 되나??
이건 아마 임의로 고르면 되는 것 맞는 것 같은데

논문에서 k의 의미는 아직 잘 모르겠음

## Reference
https://chromium.googlesource.com/external/github.com/tensorflow/tensorflow/+/r0.10/tensorflow/g3doc/tutorials/wide_and_deep/index.md

https://www.kaggle.com/code/matanivanov/wide-deep-learning-for-recsys-with-pytorch